# CADO Proof‑Kit — Certificate Verification (Presentation Notebook)

This notebook explains the **certificate check** used in v2 for Collective SLA linear algebra.
It is designed to be readable by the whole team and usable in a demo presentation.

**Goal:** verify a candidate kernel vector \(v\) by checking \(Mv=0\) over \(\mathbb{F}_2\).


## How this fits the v2 system

- **Collective LA** is expensive to duplicate across providers.
- v2 allows a **certificate**: a candidate vector \(v\) such that \(Mv=0\).
- QC checks this cheaply and records `certificate_verified=true` in the Receipt.

This makes Collective SLA feasible on a 4×3060 demo, while still being auditable.


## The certificate we verify

We need to check:

\[
Mv = 0 \quad \text{over } \mathbb{F}_2
\]

That means every row of \(M\) dotted with \(v\) is **even** (0 mod 2).
This is **much cheaper** than solving the system again.


In [ ]:
# Load tiny test vectors (demo)
import json
from pathlib import Path

base = Path('packages/cado_proofkit/test_vectors')

matrix_data = json.loads((base / 'tiny_matrix.json').read_text())
matrix_rows = matrix_data['matrix_rows']
vector_bits = matrix_data['vector_bits']

matrix_rows, vector_bits

In [ ]:
from packages.cado_proofkit.verifier_f2 import verify_f2_matrix_vector

verify_f2_matrix_vector(matrix_rows, vector_bits)

## Hash commitment (receipt field)

In v2, we store a **hash commitment** to the matrix in the Receipt. This lets auditors verify that the certificate was checked against the correct matrix version.

In [ ]:
from packages.cado_proofkit.hash_commit import hash_matrix_rows

hash_matrix_rows(matrix_rows)

## Determinism checklist (RTX 3060 demo)

To make the LA output repeatable across providers:
- Fix **seeds** and record them in the output format.
- Use **bit‑packed F2 arithmetic** (no floating‑point reductions).
- Fix **rank ordering** and **communication order** in the collective.
- Store the output in the canonical LA output format (see `packages/cado_proofkit/formats/la_output_v1.json`).


## Summary slide (what to say)

- We verify Collective LA with a **certificate**: \(Mv=0\).
- This is far cheaper than re‑running the LA stage.
- The result is deterministic and auditable across multiple providers.
- The matrix commitment + seed make it reproducible.
